In [11]:
import pandas as pd
import os

data_folder = os.path.join('..', 'data')

dataset_file = os.path.join(data_folder, 'Dataset - Pandas.csv')
real_code_file = os.path.join(data_folder, 'pandas-preprocessedcode-dataset.csv')



In [12]:
dataset = pd.read_csv(dataset_file, sep=',')
code = pd.read_csv(real_code_file, encoding='ISO-8859-1', sep=';', doublequote=True)


In [13]:
# Remove non answers
answerIds = dataset.AnswerId.unique()

print("Amount of answers in the dataset: %d" % len(answerIds))

Amount of answers in the dataset: 58


In [15]:
dataset_code = code[code.Id.isin(answerIds)]

dataset_code = dataset_code.set_index('Id')

# df['fruits'].str.split().apply(len).value_counts()

dataset_code['Code Size'] = dataset_code['PreprocessedCode'].apply(lambda x: len(x.split('\\n')))
dataset_code['Code Size']
# dataset_code['Code']

dataset_code

# dataset_code['Code Size'] = dataset_code['Code'].apply(lambda x : len(x.split('\n')))
# dataset_code.head()

,Unnamed: 0,Unnamed: 0.1,AcceptedAnswerId,AnswerCount,Body,ClosedDate,CommentCount,CommunityOwnedDate,CreationDate,FavoriteCount,...,ParentId,PostTypeId,Score,Tags,Title,ViewCount,CodeList,Code,PreprocessedCode,Code Size
Id,,,,,,,,,,,,,,,,,,,,,
10374456,195,195,NaN,NaN,<p><code>g1</code> here <em>is</em> a DataFram...,NaN,4,NaN,2012-04-29T17:50:33.950,NaN,...,10373660.0,2,241,NaN,NaN,NaN,"[""In [19]: type(g1)\r\nOut[19]: pandas.core.fr...",['In [19]: type(g1)\r\nOut[19]: pandas.core.fr...,['type(g1)\r\ng1.index\r\ng1.add_suffix(\'_Cou...,5
11287278,414,414,NaN,NaN,<p>The column names (which are strings) cannot...,NaN,12,NaN,2012-07-02T02:43:02.740,NaN,...,11285613.0,2,490,NaN,NaN,NaN,"[""df1 = df[['a','b']]\r\n"", 'df1 = df.ix[:,0:2...","[""df1 = df[['a','b']]\r\ndf1 = df.ix[:,0:2] # ...","[""df1 = df[['a','b']]\r\ndf1 = df.ix[:,0:2] # ...",4
11346337,434,434,NaN,NaN,<p>Just assign it to the <code>.columns</code>...,NaN,9,NaN,2012-07-05T14:23:27.910,NaN,...,11346283.0,2,768,NaN,NaN,NaN,"["">>> df = pd.DataFrame({'$a':[1,2], '$b': [10...","["">>> df = pd.DataFrame({'$a':[1,2], '$b': [10...",[''],1
11354850,439,439,NaN,NaN,<p>Use the rename function and refer the colum...,NaN,6,NaN,2012-07-06T01:48:15.147,NaN,...,11346283.0,2,1304,NaN,NaN,NaN,"[""df = df.rename(columns={'oldName1': 'newName...","[""df = df.rename(columns={'oldName1': 'newName...","[""df = df.rename(columns={'oldName1': 'newName...",4
11531402,499,499,NaN,NaN,"<p>Based on github issue <a href=""https://gith...",NaN,5,NaN,2012-07-17T21:52:18.983,NaN,...,11350770.0,2,267,NaN,NaN,NaN,"['df[df[\'A\'].str.contains(""hello"")]\r\n']","['df[df[\'A\'].str.contains(""hello"")]\r\n']","['df[df[\'A\'].str.contains(""hello"")]\r\n']",2
11617194,521,521,NaN,NaN,<p>The newest versions of pandas now include a...,NaN,4,NaN,2012-07-23T17:09:39.743,NaN,...,7837722.0,2,209,NaN,NaN,NaN,"['for index, row in df.iterrows():\r\n\r\n ...","['for index, row in df.iterrows():\r\n\r\n ...",['\r\n # do some logic here\r\n'],3
12065904,696,696,NaN,NaN,<p>Use the <code>isin</code> method. <code>rp...,NaN,12,NaN,2012-08-22T03:21:12.067,NaN,...,12065885.0,2,326,NaN,NaN,NaN,[],[''],[''],1
12681217,1018,1018,NaN,NaN,<p>How about something like this:</p>\r\r\r\n\...,NaN,8,NaN,2012-10-01T21:15:03.957,NaN,...,12680754.0,2,22,NaN,NaN,NaN,"[""In [55]: pd.concat([Series(row['var2'], row[...","[""In [55]: pd.concat([Series(row['var2'], row[...",[''],1
13270110,1332,1332,NaN,NaN,<p>If you have a key that is repeated for each...,NaN,1,NaN,2012-11-07T12:47:36.283,NaN,...,13269890.0,2,31,NaN,NaN,NaN,"[""from pandas import DataFrame, merge\r\ndf1 =...","[""from pandas import DataFrame, merge\r\ndf1 =...","[""from pandas import DataFrame, merge\r\ndf1 =...",6


In [18]:
l = dataset_code.loc[10374456, :]

l.PreprocessedCode.split('\\n')



["['type(g1)\\r",
 'g1.index\\r',
 "g1.add_suffix(\\'_Count\\').reset_index()\\r",
 'DataFrame({\\\'count\\\' : df1.groupby( [ "Name", "City"] ).size()}).reset_index()\\r',
 "']"]

In [19]:
dataset_code['Code Size'].describe()

count    53.000000
mean      5.547170
std       4.830058
min       1.000000
25%       2.000000
50%       4.000000
75%       7.000000
max      26.000000
Name: Code Size, dtype: float64